In [1]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession

In [3]:
#spark = SparkSession.builder \
    #.appName("Read MovieLens from HDFS") \
    #.getOrCreate()

#spark = SparkSession.builder \
    #.appName("ALS Model") \
    #.config("spark.driver.memory", "4g") \
    #.config("spark.executor.memory", "4g") \
   # .getOrCreate()

#.config("spark.hadoop.fs.defaultFS", "hdfs://bigdata-node:8088") \
spark = SparkSession.builder \
    .appName("MovieLensAnalysis") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .config("spark.sql.shuffle.partitions", "20") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.3") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/02 14:21:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Lire les fichiers

In [4]:
ratings_df = spark.read.option("header", True).csv("hdfs:///data/rating.csv")

In [ ]:
ratings_df.show(5)

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
+------+-------+------+-------------------+
only showing top 5 rows



In [ ]:
movies_df.show(5)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
+-------+--------------------+--------------------+
only showing top 5 rows



In [8]:
df_data = ratings_df.join(
    movies_df.select("movieId", "title"),
    on="movieId",
    how="left"
)

In [9]:
df_data.show(5)

+-------+------+------+-------------------+--------------------+
|movieId|userId|rating|          timestamp|               title|
+-------+------+------+-------------------+--------------------+
|      2|     1|   3.5|2005-04-02 23:53:47|      Jumanji (1995)|
|     29|     1|   3.5|2005-04-02 23:31:16|City of Lost Chil...|
|     32|     1|   3.5|2005-04-02 23:33:39|Twelve Monkeys (a...|
|     47|     1|   3.5|2005-04-02 23:32:07|Seven (a.k.a. Se7...|
|     50|     1|   3.5|2005-04-02 23:29:40|Usual Suspects, T...|
+-------+------+------+-------------------+--------------------+
only showing top 5 rows



In [10]:
df_data = df_data.withColumnsRenamed({
    'userId': 'user',
    'movieId': 'movie',
})

In [11]:
df_data.show(5)

+-----+----+------+-------------------+--------------------+
|movie|user|rating|          timestamp|               title|
+-----+----+------+-------------------+--------------------+
|    2|   1|   3.5|2005-04-02 23:53:47|      Jumanji (1995)|
|   29|   1|   3.5|2005-04-02 23:31:16|City of Lost Chil...|
|   32|   1|   3.5|2005-04-02 23:33:39|Twelve Monkeys (a...|
|   47|   1|   3.5|2005-04-02 23:32:07|Seven (a.k.a. Se7...|
|   50|   1|   3.5|2005-04-02 23:29:40|Usual Suspects, T...|
+-----+----+------+-------------------+--------------------+
only showing top 5 rows



In [12]:
from pyspark.sql.functions import col

df_data = df_data \
    .withColumn("user", col("user").cast("int")) \
    .withColumn("movie", col("movie").cast("int")) \
    .withColumn("rating", col("rating").cast("float")) \
    .withColumn("timestamp", col("timestamp").cast("int")) \
    .withColumn("title", col("title").cast("string"))

In [13]:
df_data_train, df_data_test = df_data.randomSplit([0.8, 0.2], 42)

In [14]:
# import ALS model from spark
from pyspark.ml.recommendation import ALS

# initialise the model 
als = ALS(maxIter=10, regParam=0.1, rank=10, userCol="user", itemCol="movie", 
          ratingCol="rating", coldStartStrategy="drop", nonnegative=True)

# train model 
als_model = als.fit(df_data_train)

25/05/02 14:22:16 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
                                                                                

In [15]:
dfs_preds = als_model.transform(df_data_test)

In [16]:
dfs_preds.show(5)

[Stage 114:==================================================>    (11 + 1) / 12]

+-----+----+------+---------+--------------------+----------+
|movie|user|rating|timestamp|               title|prediction|
+-----+----+------+---------+--------------------+----------+
|   47|  95|   3.0|     NULL|Seven (a.k.a. Se7...|  3.078373|
|  262|  95|   3.0|     NULL|Little Princess, ...|  3.438147|
|  446|  95|   3.0|     NULL|Farewell My Concu...| 3.5873995|
|  592|  95|   3.0|     NULL|       Batman (1989)|  2.504309|
|  708|  95|   4.0|     NULL|Truth About Cats ...|  3.118482|
+-----+----+------+---------+--------------------+----------+
only showing top 5 rows



In [17]:
als_model.save("hdfs:///data/als_prediction_rating")

Py4JJavaError: An error occurred while calling o195.save.
: java.io.IOException: Path hdfs:///data/als_prediction_rating already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(Abstr

In [18]:
from pyspark.ml.recommendation import ALSModel

als_model_loaded = ALSModel.load("hdfs:///data/als_prediction_rating")

In [19]:
dfs_preds_loaded = als_model_loaded.transform(df_data_test)

In [20]:
dfs_preds_loaded.show(5)

[Stage 147:>                                                        (0 + 1) / 1]

+-----+----+------+---------+----------------+----------+
|movie|user|rating|timestamp|           title|prediction|
+-----+----+------+---------+----------------+----------+
|    1|   8|   4.0|     NULL|Toy Story (1995)| 4.1200004|
|    1|  13|   4.0|     NULL|Toy Story (1995)|    3.5436|
|    1|  16|   3.0|     NULL|Toy Story (1995)| 3.7518215|
|    1|  31|   3.0|     NULL|Toy Story (1995)| 3.0558991|
|    1|  58|   5.0|     NULL|Toy Story (1995)|   4.34349|
+-----+----+------+---------+----------------+----------+
only showing top 5 rows



In [21]:
df_data_test.show()

[Stage 149:>                                                        (0 + 1) / 1]

+-----+----+------+---------+----------------+
|movie|user|rating|timestamp|           title|
+-----+----+------+---------+----------------+
|    1|   8|   4.0|     NULL|Toy Story (1995)|
|    1|  13|   4.0|     NULL|Toy Story (1995)|
|    1|  16|   3.0|     NULL|Toy Story (1995)|
|    1|  31|   3.0|     NULL|Toy Story (1995)|
|    1|  58|   5.0|     NULL|Toy Story (1995)|
|    1|  80|   3.0|     NULL|Toy Story (1995)|
|    1|  96|   3.5|     NULL|Toy Story (1995)|
|    1| 114|   3.0|     NULL|Toy Story (1995)|
|    1| 134|   4.0|     NULL|Toy Story (1995)|
|    1| 135|   4.0|     NULL|Toy Story (1995)|
|    1| 136|   5.0|     NULL|Toy Story (1995)|
|    1| 140|   4.0|     NULL|Toy Story (1995)|
|    1| 144|   3.0|     NULL|Toy Story (1995)|
|    1| 155|   2.5|     NULL|Toy Story (1995)|
|    1| 172|   5.0|     NULL|Toy Story (1995)|
|    1| 204|   5.0|     NULL|Toy Story (1995)|
|    1| 232|   3.0|     NULL|Toy Story (1995)|
|    1| 296|   5.0|     NULL|Toy Story (1995)|
|    1| 318| 

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator

# initiate evaluator
eval = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
# calculate rmse
rmse = eval.evaluate(dfs_preds_loaded)

# reset metric and evaluate mse, R2, mae & explained variance
r2 = eval.evaluate(dfs_preds_loaded, {eval.metricName: 'r2'})
mae = eval.evaluate(dfs_preds_loaded, {eval.metricName: 'mae'})
var = eval.evaluate(dfs_preds_loaded, {eval.metricName: 'var'})


print(
    "RMSE score = {}".format(rmse),
    "R2 score = {}".format(r2),
    "MAE score = {}".format(mae),
    "Explained variance score = {}".format(var),
    sep="\n"
)

[Stage 168:================================>                       (7 + 5) / 12]

RMSE score = 0.8157736828362929
R2 score = 0.39869052585871656
MAE score = 0.6375367684273177
Explained variance score = 0.4086564056899666
